In [1]:
import syft as sy
import torch as t

In [2]:
hook = sy.TorchHook(t)

In [3]:
bob = sy.VirtualWorker(hook,id="bob")

In [5]:
x = t.tensor([1,2,3,4,5]).send(bob)
y = t.tensor([1,2,3,4,5]).send(bob)

In [9]:
x

(Wrapper)>[PointerTensor | me:10794735732 -> bob:84255456333]

In [10]:
y

(Wrapper)>[PointerTensor | me:37239912144 -> bob:74281964209]

In [11]:
z = x + y

In [12]:
z

(Wrapper)>[PointerTensor | me:54410037431 -> bob:61829269040]

In [13]:
z = z.get()
z

tensor([ 2,  4,  6,  8, 10])

In [14]:
z = t.add(x,y)
z

(Wrapper)>[PointerTensor | me:46284848438 -> bob:13388972260]

In [15]:
z = z.get()
z

tensor([ 2,  4,  6,  8, 10])

### BackPropagaton

In [20]:
x = t.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = t.tensor([1.,1,1,1,1], requires_grad=True).send(bob)
#Just to create floating tensors I replace the initial value as 1.

In [22]:
z = (x + y).sum() #to get a single value

In [23]:
z.backward()

(Wrapper)>[PointerTensor | me:93614312900 -> bob:34467330400]

In [24]:
x = x.get()
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [26]:
x.grad

tensor([1., 1., 1., 1., 1.])

In [25]:
y = y.get()
y

tensor([1., 1., 1., 1., 1.], requires_grad=True)

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [32]:
inp = t.tensor([[1.,1],[0,1],[1,0],[0,0]],requires_grad=True).send(bob)

In [33]:
inp

(Wrapper)>[PointerTensor | me:18284111538 -> bob:1042399836]

In [34]:
target = t.tensor([[1.],[1],[0],[0]],requires_grad=True).send(bob)

In [35]:
target

(Wrapper)>[PointerTensor | me:9766546812 -> bob:95217787191]

In [45]:
weights = t.tensor([[0.],[0.]],requires_grad=True).send(bob)

In [46]:
weights

(Wrapper)>[PointerTensor | me:99575996462 -> bob:98975765157]

## Forward Pass

In [47]:
pred = inp.mm(weights)
pred

(Wrapper)>[PointerTensor | me:55898902952 -> bob:32223548306]

In [48]:
loss = ((pred - target)**2).sum()
loss

(Wrapper)>[PointerTensor | me:66830134225 -> bob:60037171818]

In [49]:
#Update
loss.backward()
weights.data.sub_(weights.grad * 0.1)
weights.grad *= 0 #So they dont accumulate over time

(Wrapper)>[PointerTensor | me:90854740965 -> bob:51738509419]

In [50]:
print(loss.get().data)

tensor(2.)


In [51]:
#looP it
for i in range(10):
    pred = inp.mm(weights)
    
    loss = ((pred - target)**2).sum()
    loss.backward()
    
    weights.data.sub_(weights.grad * 0.1)
    weights.grad *= 0
    
    print(loss.get().data)

tensor(2.)
tensor(0.2400)
tensor(0.1248)
tensor(0.0753)
tensor(0.0474)
tensor(0.0302)
tensor(0.0193)
tensor(0.0124)
tensor(0.0079)
tensor(0.0051)


# Lesson: Garbage Collection and Common Errors

In [78]:
bob = bob.clear_objects()

In [79]:
bob._objects

{}

In [89]:
x = t.tensor([1,2,3,4,5]).send(bob)

In [90]:
print(x)

(Wrapper)>[PointerTensor | me:61371793303 -> bob:30736919738]


In [91]:
bob._objects

{30736919738: tensor([1, 2, 3, 4, 5])}

In [92]:
del x

In [93]:
bob._objects

{}

In [94]:
x = t.tensor([1,2,3,4,5]).send(bob)

In [95]:
x.child.garbage_collect_data

True

In [99]:
bob._objects

{33276634139: tensor([1, 2, 3, 4, 5])}

In [100]:
x = "asdf"

In [101]:
bob._objects

{33276634139: tensor([1, 2, 3, 4, 5])}

In [105]:
del x

In [107]:
bob._objects #So now it didn't delete object of bob WHY ??
#Coz we changed what x was pointing to by x="asdf"
#So now we've to do this

{33276634139: tensor([1, 2, 3, 4, 5])}

In [110]:
bob.clear_objects()

<VirtualWorker id:bob #objects:0>

In [111]:
bob._objects

{}

In [112]:
for i in range(1000):
    x = t.tensor([1,2,3,4,5]).send(bob)

In [115]:
bob._objects#Why not 1000 objects?
#Coz everytime it reassigned x and deleted the tensor that was returned which caused it to be garbage 
#collected from remote machine
#Its a realy imp feature that if garbage is collected then we delete its reference also .
#otherwise for just a simple forward pass we woulf have a thousands of references collected

{89224130609: tensor([1, 2, 3, 4, 5])}

In [116]:
x = t.tensor([1,2,3,4,5]).send(bob)
y = t.tensor([1,2,3,4,5])

In [118]:
#z = x + y
#If u uncomment  z = x+y u'll encounter this error
'''
You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

'''

'\nYou tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.\n\n'

In [119]:
alice = sy.VirtualWorker(hook, id="alice")

In [123]:
x = t.tensor([1,2,3,4,5]).send(bob)
y = t.tensor([1,2,3,4,5]).send(alice)
#One tensor is on bob and another on alice

In [124]:
#z = x + y
#If u uncomment  z = x+y u'll encounter this error
'''
You tried to call __add__ involving two tensors which are not on the same machine! One tensor is on <VirtualWorker id:bob #objects:3> while the other is on <VirtualWorker id:alice #objects:1>. Use a combination of .move(), .get(), and/or .send() to co-locate them to the same machine.

'''

'\nYou tried to call __add__ involving two tensors which are not on the same machine! One tensor is on <VirtualWorker id:bob #objects:3> while the other is on <VirtualWorker id:alice #objects:1>. Use a combination of .move(), .get(), and/or .send() to co-locate them to the same machine.\n\n'